In [20]:
# pdp package path
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

# imports
import random
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [21]:
## Define problems with sizes 10, 15, 20
# Each with an id 1, 2, 3, ... where (id // 3) * 5 + 10 = number of requests

items_list = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(300, 400),
]

# 2 small compartments and 2 large compartments
vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 400),
    ]
)

distance_matrix = [
    [0, 1, 1, 1, 11, 11, 11],
    [1, 0, 1, 2, 10, 11, 12],
    [1, 1, 0, 1, 11, 10, 11],
    [1, 2, 1, 0, 12, 11, 10],
    [11, 10, 11, 12, 0, 1, 2],
    [11, 11, 10, 11, 1, 0, 1],
    [11, 12, 11, 10, 2, 1, 0],
]

problem = TwoDimensionalProblem(items=items_list, vehicle=vehicle, distance_matrix=distance_matrix)
modelled_problem = ModelledTwoDimensionalProblem(items_list, vehicle, distance_matrix=distance_matrix)


modelled_problem.create_model()
modelled_problem.apply_constraints()
modelled_problem.set_model_objective(OptimizationObjective.CHEAPEST_ROUTE)


In [22]:
# Initial, quick solutions by gurobi
initial_solution_time_limit = 0.02
modelled_problem.solve(time_limit=initial_solution_time_limit, write_model=True)
initial_solution = modelled_problem.extract_solution()

Set parameter TimeLimit to value 0.02
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 275 rows, 93 columns and 1127 nonzeros
Model fingerprint: 0x6faf2e7a
Variable types: 0 continuous, 93 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 90 rows and 35 columns
Presolve time: 0.00s
Presolved: 185 rows, 58 columns, 680 nonzeros
Variable types: 0 continuous, 58 integer (40 binary)
Found heuristic solution: objective 64.0000000

Root relaxation: objective 8.000000e+00, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.00000    0   17   64.00000    8.00000  87.5%     -    0s



In [27]:
initial_solution
# problem.evaluate_solution(initial_solution)

Solution(
	order=0, 1, 4, 2, 5, 3, 6, 
	step: 0 vertex: 0 	0, 0
	step: 1 vertex: 1 	400, 0
	step: 2 vertex: 4 	0, 0
	step: 3 vertex: 2 	400, 0
	step: 4 vertex: 5 	0, 0
	step: 5 vertex: 3 	400, 0
	step: 6 vertex: 6 	0, 0
)

In [24]:
import time

In [25]:
def get_gap(exact_val, heuristic_val):
    zp = heuristic_val
    zd = exact_val

    return abs(zp - zd) / abs(zp)

In [29]:
time_limit = 3
solver = LNS(problem, min_destruction_degree=0.35, max_destruction_degree=0.35, initial_solution=initial_solution, time_limit=time_limit)
solver.search()

Terminating LNS in 2867 iterations
Total time: 3.04 Best objective: 15 found in 4 iterations in 0.03 s



Solution(
	order=0, 3, 2, 1, 4, 5, 6, 
	step: 0 vertex: 0 	0, 0
	step: 1 vertex: 3 	0, 300
	step: 2 vertex: 2 	400, 300
	step: 3 vertex: 1 	800, 300
	step: 4 vertex: 4 	400, 300
	step: 5 vertex: 5 	0, 300
	step: 6 vertex: 6 	0, 0
)

In [30]:
solver.solutions_cache

[SearchSolution(iteration=-1, solution=Solution(
 	order=0, 1, 4, 2, 5, 3, 6, 
 	step: 0 vertex: 0 	0, 0
 	step: 1 vertex: 1 	400, 0
 	step: 2 vertex: 4 	0, 0
 	step: 3 vertex: 2 	400, 0
 	step: 4 vertex: 5 	0, 0
 	step: 5 vertex: 3 	400, 0
 	step: 6 vertex: 6 	0, 0
 ), objective=53, time_to_find=0, iteration_time=0, time_found=1655684546.0082097),
 SearchSolution(iteration=1, solution=Solution(
 	order=0, 1, 4, 2, 3, 6, 5, 
 	step: 0 vertex: 0 	0, 0
 	step: 1 vertex: 1 	400, 0
 	step: 2 vertex: 4 	0, 0
 	step: 3 vertex: 2 	400, 0
 	step: 4 vertex: 3 	800, 0
 	step: 5 vertex: 6 	400, 0
 	step: 6 vertex: 5 	0, 0
 ), objective=34, time_to_find=0.022953033447265625, iteration_time=0.022698640823364258, time_found=1655684546.0312104),
 SearchSolution(iteration=2, solution=Solution(
 	order=0, 2, 1, 4, 5, 3, 6, 
 	step: 0 vertex: 0 	0, 0
 	step: 1 vertex: 2 	400, 0
 	step: 2 vertex: 1 	800, 0
 	step: 3 vertex: 4 	400, 0
 	step: 4 vertex: 5 	0, 0
 	step: 5 vertex: 3 	400, 0
 	step: 6 vertex: